In [ ]:
import requests
import pandas as pd
from datetime import date ,timedelta

polygon_api_key = open("polygon_io_api_key.txt").read()

ticker = ''.upper()

today = datetime.date.today()

expiration = datetime.date.today() + timedelta(days=7)


In [ ]:
#Identifies the last day's stocks aggregate results
#Current free version of the stocks plan only includes up to the last previous day
last_stock_url = f'https://api.polygon.io/v2/aggs/ticker/{ticker}/prev?adjusted=true&apiKey={polygon_api_key}'
last_stock_request = requests.get(last_stock_url)
last_stock_result = last_stock_request.json()

last_stock_df = pd.DataFrame(last_stock_result['results'])
last_close = last_stock_df['c'][0]

#Identify ATM call
call_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?strike_price.gte={last_close}&expiration_date.lte={expiration}&contract_type=call&order=asc&limit=1&sort=strike_price&apiKey={polygon_api_key}'
call_request = requests.get(call_url)
call_result = call_request.json()

#Identify ATM put
put_url = f'https://api.polygon.io/v3/snapshot/options/{ticker}?strike_price.lte={last_close}&contract_type=put&expiration_date.lte={expiration}&order=desc&limit=1&sort=strike_price&apiKey={polygon_api_key}'
put_request = requests.get(put_url)
put_result = put_request.json()

call_atm = call_result['results'][0]['day']['previous_close']
put_atm = put_result['results'][0]['day']['previous_close']

expected_move_straddle = (call_atm + put_atm) * .85

print(expected_move_straddle)